In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
import japanize_matplotlib


# 入出力ファイルの設定
INPUT_FILE_NAME = 'data.csv'
OUTPUT_IMAGE_BALANCE = 'covariate_balance.png'

# フォント設定
plt.rcParams['font.size'] = 12

# カラム名の英語変換マッピング
rename_dict = {
    # 属性
    "年齢を教えてください。": "Age",
    "性別を教えてください。": "Gender",
    
    # 味覚項目
    "甘味をどの程度感じましたか？（１／５）": "Sweetness",
    "うま味をどの程度感じましたか？（２／５）": "Umami",
    "苦味をどの程度感じましたか？（３／５）": "Bitterness",
    "酸味をどの程度感じましたか？（４／５）": "Sourness",
    "塩味をどの程度感じましたか？（５／５）": "Saltiness",
    
    # 視覚項目
    "華やかさをどの程度感じましたか？（１／５）": "Floral",
    "開放感をどの程度感じましたか？（２／５）": "Openness",
    "爽やかさをどの程度感じましたか？（３／５）": "Freshness",
    "賑わい感をどの程度感じましたか？（４／５）": "Liveliness",
    "賑い感をどの程度感じましたか？（４／５）": "Liveliness", 
    "高揚感をどの程度感じましたか？（５／５）": "Excitement",
    
    # 目的変数
    "クラフトビール「白蒲田」をまた飲んでみたいと思いますか？": "Drink_Again"
}

# 目的変数
TARGET_VARIABLE = "Drink_Again"

# 検証したい介入変数のリスト
# ここに、前の分析(Lassoや順序ロジスティック)で重要と判明した変数を入力
TREATMENT_VARIABLES = [
    "Floral",      # 華やかさ
    "Umami",       # うま味
    "Excitement"   # 高揚感（もしあれば）
]

# 介入あり（高評価）」とみなす閾値
# 4点以上（4と5）を「介入あり(1)」、3点以下を「介入なし(0)」として扱います
TREATMENT_THRESHOLD = 4

# ==========================================
# 関数定義セクション
# ==========================================

def calculate_ipw(df, treatment_col, target_col, covariates):
    """
    傾向スコアを用いてIPW（ATT: 処置群における平均処置効果）を算出する関数
    """
    # 処置変数(T)の2値化: 閾値以上を1、未満を0にする
    T = (df[treatment_col] >= TREATMENT_THRESHOLD).astype(int)
    Y = df[target_col]
    X = df[covariates]
    
    # 傾向スコアの算出 (ロジスティック回帰)
    # データの標準化
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    model = LogisticRegression(random_state=42, solver='lbfgs', max_iter=1000)
    model.fit(X_scaled, T)
    
    # 傾向スコア (e) = P(T=1|X)
    ps_score = model.predict_proba(X_scaled)[:, 1]
    
    # ATT用重みの計算
    # 処置群(T=1)は重み1、対照群(T=0)は e/(1-e) の重みをつける
    # これにより、対照群の分布を処置群に似せる（擬似的な対照群を作る）
    weights = T + (1 - T) * ps_score / (1 - ps_score)
    
    # 因果効果 (ATT) の推定
    # 重み付き平均の差
    mean_treated = np.average(Y[T==1], weights=weights[T==1])
    mean_control = np.average(Y[T==0], weights=weights[T==0])
    att = mean_treated - mean_control
    
    # 単純な平均の差（比較用）
    simple_diff = Y[T==1].mean() - Y[T==0].mean()
    
    return {
        'ATT': att,
        'Simple_Diff': simple_diff,
        'Mean_Treated': mean_treated,
        'Mean_Control_Weighted': mean_control,
        'Weights': weights,
        'PS_Score': ps_score,
        'T_binary': T,
        'Model': model
    }

def plot_covariate_balance(df, treatment_col, covariates, weights):
    """
    共変量バランス（SMD: 標準化平均差）をプロットする関数 (Love Plot)
    """
    T = (df[treatment_col] >= TREATMENT_THRESHOLD).astype(int)
    X = df[covariates]
    
    # 調整前（Unadjusted）のSMD
    mean_diff_unadj = X[T==1].mean() - X[T==0].mean()
    pooled_sd = np.sqrt((X[T==1].var() + X[T==0].var()) / 2)
    smd_unadj = mean_diff_unadj / pooled_sd
    
    # 調整後（Adjusted by IPW）のSMD
    # 重み付き平均と分散を計算するのは複雑なため、簡易的に重み付き平均を使用
    mean_t_w = np.average(X[T==1], axis=0, weights=weights[T==1])
    mean_c_w = np.average(X[T==0], axis=0, weights=weights[T==0])
    smd_adj = (mean_t_w - mean_c_w) / pooled_sd
    
    # プロット用データフレーム
    balance_df = pd.DataFrame({
        'Variable': covariates,
        'Unadjusted': smd_unadj.abs(),
        'Adjusted': smd_adj.abs()
    })
    
    # プロット
    plt.figure(figsize=(10, len(covariates) * 0.4 + 2))
    plt.plot(balance_df['Unadjusted'], balance_df['Variable'], 'o', color='red', label='Unadjusted', alpha=0.6)
    plt.plot(balance_df['Adjusted'], balance_df['Variable'], 'o', color='blue', label='Adjusted (IPW)', alpha=0.8)
    plt.axvline(x=0.1, color='black', linestyle='--', linewidth=0.8, label='Threshold (0.1)')
    plt.title(f"Covariate Balance: Treatment = {treatment_col}")
    plt.xlabel("Absolute Standardized Mean Difference (SMD)")
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.5)
    plt.show()

# ==========================================
# メイン処理
# ==========================================

# データの読み込みと前処理
try:
    df = pd.read_csv(INPUT_FILE_NAME, encoding='shift_jis')
    print(f"データ読み込み成功: {INPUT_FILE_NAME}")
except FileNotFoundError:
    print(f"エラー: ファイル '{INPUT_FILE_NAME}' が見つかりません。")
    # ダミーデータ生成
    df = pd.DataFrame(np.random.randint(1, 6, (50, 13)), columns=list(rename_dict.keys()))
    df['性別を教えてください。'] = np.random.choice(['男性', '女性'], 50)

# リネーム
df_renamed = df.rename(columns=rename_dict)
valid_cols = [col for col in rename_dict.values() if col in df_renamed.columns]
df_analysis = df_renamed[valid_cols]

# 数値変換
num_cols = [c for c in df_analysis.columns if c != 'Gender']
for col in num_cols:
    df_analysis[col] = pd.to_numeric(df_analysis[col], errors='coerce')
df_analysis = df_analysis.dropna()

# 性別のダミー変数化
df_analysis = pd.get_dummies(df_analysis, columns=['Gender'], drop_first=True)
print("データ前処理完了。")


# IPWによる因果効果の推定 (ループ処理)
results_list = []

print(f"\n{'='*60}")
print(f" IPW (Inverse Probability Weighting) による因果効果の推定")
print(f" ターゲット(Y): {TARGET_VARIABLE}")
print(f" 閾値: {TREATMENT_THRESHOLD} 以上を「介入あり(高評価)」とする")
print(f"{'='*60}\n")

for treatment_var in TREATMENT_VARIABLES:
    if treatment_var not in df_analysis.columns:
        print(f"※ 変数 {treatment_var} がデータにないためスキップします。")
        continue

    # 共変量（Covariates）の定義: 介入変数と目的変数 以外のすべて
    # 性別(Gender_男性)や年齢(Age)もここに含まれます
    covariates = [c for c in df_analysis.columns if c not in [TARGET_VARIABLE, treatment_var]]
    
    # IPW計算実行
    res = calculate_ipw(df_analysis, treatment_var, TARGET_VARIABLE, covariates)
    
    # 結果の保存
    results_list.append({
        'Treatment': treatment_var,
        'ATT (因果効果)': res['ATT'],
        'Simple Diff (単純差)': res['Simple_Diff']
    })
    
    print(f"■ 介入変数: {treatment_var}")
    print(f"   - 単純なリピート意向の差: {res['Simple_Diff']:.3f} 点")
    print(f"   - 補正後の因果効果(ATT):  {res['ATT']:.3f} 点")
    print(f"   (この要素を高評価した人は、しなかった場合に比べてリピート意向が {res['ATT']:.2f} 点上がったと推定される)")
    
    # バランスチェックのプロット
    plot_covariate_balance(df_analysis, treatment_var, covariates, res['Weights'])
    print("-" * 60)

# 3. 全結果のまとめ表示
results_df = pd.DataFrame(results_list)
print("\n【推定結果まとめ】")
display(results_df.round(3))

# グラフ化
plt.figure(figsize=(10, 6))
x = np.arange(len(results_df))
width = 0.35

plt.bar(x - width/2, results_df['Simple Diff (単純差)'], width, label='Simple Diff', color='lightgray')
plt.bar(x + width/2, results_df['ATT (因果効果)'], width, label='ATT (Causal Effect)', color='royalblue')

plt.xticks(x, results_df['Treatment'])
plt.ylabel("Effect on Repeat Intention (Points)")
plt.title("Causal Effect (ATT) of Key Factors on Repeat Intention")
plt.legend()
plt.grid(axis='y', linestyle='--', alpha=0.5)
plt.show()